In [3]:
# import PyLTSpice.raw_read
import numpy as np
from PyLTSpice import SimCommander, RawRead

high_side = True
low_side = False

def frequencytoltpulse(frequency, side: bool):
    deadband = 8  # ns
    period = 1 / (frequency / 1E9)  # Convert to Gigahertz to ensure the period will be in nanoseconds
    if side == low_side:
        return f"PULSE(3.3 0 0n 1n 1n {period + (deadband * 2)}n {period * 2}n)"
    else:
        return f"PULSE(0 3.3 {deadband}n 1n 1n {period}n {period * 2}n)"

fsw = 100E3
L = 10E-6
C = 47E-6

# select spice model
LTC = SimCommander("LTFiles/EPC23102_Mine.asc")
# set default arguments
LTC.set_component_value('VHSin', frequencytoltpulse(fsw, high_side))
LTC.set_component_value('VLSin', frequencytoltpulse(fsw, low_side))

Using LTspice installed in : 'C:\Users\treys\AppData\Local\Programs\ADI\LTspice\LTspice.exe' 
Creating Netlist
The Netlist was successfully created


In [4]:
run_netlist_file = f'LTFiles/F{fsw}_L{L}_C{C}'
LTC.run(run_filename=f'{run_netlist_file}.net')

Wed Apr 26 11:23:33 2023 : Starting simulation 1


<RunTask(sim1, started 10420)>

In [10]:
LTR = RawRead(f'{run_netlist_file}.raw')
# print(LTR.get_trace_names())

class LTTraceData:
    def __init__(self, raw_file, trace_name: str):
        raw_data = raw_file.get_trace(trace_name)
        self.data = np.array(raw_data)

    # @property
    # def data(self):
    #     return self.data
    #
    # @data.setter
    # def data(self, value):
    #     self._data = value

    @property
    def average(self):
        return np.average(self.data)

    @property
    def peak_to_peak(self):
        return np.ptp(self.data)

    @property
    def rms(self):
        return np.sqrt(np.mean(self.data**2))

    @property
    def min(self):
        return np.min(self.data)

    @property
    def max(self):
        return np.max(self.data)

v_in = LTTraceData(LTR, 'V(vin)')
v_out = LTTraceData(LTR, 'V(vout)')
i_in = LTTraceData(LTR, 'I(Vvin)')
i_out = LTTraceData(LTR, 'I(I1)')

print(v_out.average)
print(v_out.peak_to_peak)

print(f'Eff: {abs(v_out.average * i_out.average) / abs(v_in.average * i_in.average) * 100}')

Reading file with encoding  utf_16_le
File contains 31 traces, reading 31
Binary RAW file with Normal access
5.0156527
0.9329524
Eff: 86.00107431411743
